In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from integral_timber_joints.planning import load_pddlstream

Using pddlstream from C:\Users\harry\Documents\code_ws\pb_ws\coop_assembly\external\pddlstream\pddlstream\__init__.py


## Parse ITJ process

In [3]:
import os
from termcolor import cprint
import pybullet_planning as pp
from integral_timber_joints.planning.parsing import parse_process, save_process_and_movements, \
    get_process_path, save_process

In [16]:
# parse options
design_dir = '210916_SymbolicPlanning' # '210419_AnticlasticShelter' # '210605_ScrewdriverTestProcess' # 210419_AnticlasticShelter
problem = 'nine_pieces_process.json' #'shelter_process.json' # 'shelter_process.json' pavilion_process.json' # 'twelve_pieces_process.json'
problem_subdir = '.'

In [17]:
process = parse_process(design_dir, problem, subdir=problem_subdir)

# Double check entire solution is valid
for beam_id in process.assembly.sequence:
    if not process.dependency.beam_all_valid(beam_id):
        print('Yay')
        process.dependency.compute_all(beam_id)
        assert process.dependency.beam_all_valid(beam_id)

Process json parsed from c:\users\harry\dropbox (mit)\code_ws_dropbox\itj_ws\integral_timber_joints\external\itj_design_study\210916_SymbolicPlanning\nine_pieces_process.json


In [19]:
assembly_wcf_inclampapproach = process.get_gripper_t0cp_for_beam_at('b3', 'assembly_wcf_inclampapproach')

In [20]:
assembly_wcf_inclampapproach

Frame(Point(18548.392, 7547.165, 2514.208), Vector(0.000, 0.391, -0.920), Vector(-0.000, 0.920, 0.391))

## Construct PDDL problem

In [6]:
from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem

In [7]:
debug = 0
verbose = 1
use_partial_order = 1
reset_to_home = 0

In [8]:
pddlstream_problem = get_pddlstream_problem(process, use_partial_order=use_partial_order, debug=debug, 
                                            reset_to_home=reset_to_home)

In [9]:
print('Init:', pddlstream_problem.init)
print()
print('Goal:', pddlstream_problem.goal)

Init: [('RobotConf', Conf-Home), ('RobotAtConf', Conf-Home), ('CanFreeMove',), ('RobotToolChangerEmpty',), ('Element', 'b0'), ('AtRack', 'b0'), ('IsElement', 'b0'), ('Grounded', 'b0'), ('Element', 'b1'), ('AtRack', 'b1'), ('IsElement', 'b1'), ('Grounded', 'b1'), ('Element', 'b2'), ('AtRack', 'b2'), ('IsElement', 'b2'), ('Element', 'b3'), ('AtRack', 'b3'), ('IsElement', 'b3'), ('Element', 'b5'), ('AtRack', 'b5'), ('IsElement', 'b5'), ('Grounded', 'b5'), ('Element', 'b6'), ('AtRack', 'b6'), ('IsElement', 'b6'), ('Element', 'b7'), ('AtRack', 'b7'), ('IsElement', 'b7'), ('Grounded', 'b7'), ('Element', 'b8'), ('AtRack', 'b8'), ('IsElement', 'b8'), ('Element', 'b4'), ('AtRack', 'b4'), ('IsElement', 'b4'), ('Joint', 'b0', 'b2'), ('Joint', 'b2', 'b0'), ('NoToolAtJoint', 'b0', 'b2'), ('NoToolAtJoint', 'b2', 'b0'), ('Joint', 'b0', 'b3'), ('Joint', 'b3', 'b0'), ('NoToolAtJoint', 'b0', 'b3'), ('NoToolAtJoint', 'b3', 'b0'), ('Joint', 'b0', 'b6'), ('Joint', 'b6', 'b0'), ('NoToolAtJoint', 'b0', 'b6')

In [10]:
from integral_timber_joints.assembly.beam_assembly_method import BeamAssemblyMethod

beam_seq = process.assembly.sequence
for e in beam_seq:
    if process.assembly.get_assembly_method(e) == BeamAssemblyMethod.GROUND_CONTACT:
        print('Grounded: ', e)
for j in process.assembly.joint_ids():
    print(j)

Grounded:  b0
Grounded:  b1
Grounded:  b5
Grounded:  b7
('b0', 'b2')
('b0', 'b3')
('b0', 'b6')
('b1', 'b2')
('b1', 'b3')
('b1', 'b8')
('b2', 'b0')
('b2', 'b1')
('b3', 'b0')
('b3', 'b1')
('b4', 'b5')
('b4', 'b7')
('b5', 'b4')
('b5', 'b6')
('b6', 'b0')
('b6', 'b5')
('b7', 'b4')
('b7', 'b8')
('b8', 'b1')
('b8', 'b7')


## Solve

In [10]:
from pddlstream.algorithms.downward import set_cost_scale, parse_action
from pddlstream.algorithms.meta import solve
from pddlstream.utils import INF
from pddlstream.language.constants import print_plan, is_plan

from integral_timber_joints.planning.pddlstream_definitions.problem import get_pddlstream_problem

pddlstream_problem = get_pddlstream_problem(process, use_partial_order=1, debug=True,
                                            reset_to_home=1)

# print('Init:', pddlstream_problem.init)
print()
print('Goal:', pddlstream_problem.goal)
print()

costs = True
set_cost_scale(1)

solution = solve(pddlstream_problem, algorithm='incremental', #incremental
                 max_time=60,
                 unit_costs=True,
                 max_planner_time=300, 
                 debug=0, verbose=0)

plan, cost, evaluations = solution
plan_success = is_plan(plan)
cprint('Planning {}'.format('succeeds' if plan_success else 'fails'), 'green' if plan_success else 'red')

print('-'*10)
print_plan(plan)


Goal: ('and', ('Assembled', 'b0'), ('Assembled', 'b1'), ('Assembled', 'b2'), ('Assembled', 'b3'), ('Assembled', 'b5'), ('Assembled', 'b6'), ('Assembled', 'b7'), ('Assembled', 'b8'), ('Assembled', 'b4'), ('AtRack', 'c1'), ('AtRack', 'c2'), ('AtRack', 'c3'), ('AtRack', 'c4'), ('AtRack', 'g1'), ('AtRack', 'g2'), ('AtRack', 'g3'), ('RobotAtConf', Conf-Home))

Iteration: 1 | Complexity: 0 | Calls: 0 | Evaluations: 115 | Solved: False | Cost: inf | Search Time: 0.003 | Sample Time: 0.000 | Time: 0.003
Iteration: 2 | Complexity: 1 | Calls: 33 | Evaluations: 245 | Solved: False | Cost: inf | Search Time: 0.813 | Sample Time: 0.003 | Time: 0.816
Summary: {complexity: 2, cost: 56.000, evaluations: 245, iterations: 2, length: 56, run_time: 2.376, sample_time: 0.003, search_time: 2.373, solutions: 1, solved: True, timeout: False}
Planning succeeds
----------
 1) pick_gripper_from_rack g1 @conf126 @conf226 @traj27
 2) pick_element_from_rack b0 @conf10 @conf20 @traj1 g1
 3) place_element_on_structu

In [75]:
from pddlstream.algorithms.downward import is_valid_plan, apply_action, task_from_domain_problem, get_problem, \
    get_action_instances
from pddlstream.algorithms.algorithm import parse_problem
from pddlstream.algorithms.constraints import PlanConstraints

# is_valid_plan(pddlstream_problem.init, plan)
# task = task_from_domain_problem(pddlstream_problem, get_problem(evaluations, goal_expression, domain, unit_costs=True))

unit_costs = True
evaluations, goal_expression, domain, externals = parse_problem(
    pddlstream_problem, constraints=PlanConstraints(), unit_costs=unit_costs)

In [76]:
from pddlstream.algorithms.refinement import optimistic_process_streams
from pddlstream.algorithms.scheduling.recover_streams import evaluations_from_stream_plan

results, exhausted = optimistic_process_streams(evaluations, externals, complexity_limit=INF, max_effort=None)
exh_evaluations = evaluations_from_stream_plan(evaluations, results, max_effort=None)

task = task_from_domain_problem(domain, get_problem(exh_evaluations, goal_expression, domain, unit_costs))

In [77]:
instantiated = instantiate_task(task)


Generating Datalog program... [0.000s CPU, 0.003s wall-clock]
Normalizing Datalog program...
Normalizing Datalog program: [0.000s CPU, 0.002s wall-clock]
Preparing model... [0.016s CPU, 0.005s wall-clock]
Generated 74 rules.
Computing model... [0.125s CPU, 0.131s wall-clock]
2180 relevant atoms
171 auxiliary atoms
2351 final queue length
7301 total queue pushes
Completing instantiation... [0.078s CPU, 0.069s wall-clock]
Infeasible: False
Instantiation time: 0.230s
Instantiated frequencies:
Atoms: {alltoolatjoints: 30, assembled: 9, atrack: 16, attached: 16, canfreemove: 1, connected: 30, new-axiom@0: 8, new-axiom@1: 20, new-axiom@2: 9, notoolatjoint: 20, robotatconf: 9, robotgripperempty: 1, robottoolchangerempty: 1, toolatjoint: 140, toolnotoccupiedonjoint: 7}
Actions: {move: 81, pick_clamp_from_joint: 80, pick_clamp_from_rack: 4, pick_element_from_rack: 27, pick_gripper_from_rack: 3, place_clamp_at_joint: 160, place_element_on_structure: 39, place_tool_at_rack: 11}
Axioms: {alltoola

In [63]:
a = plan[0]
type(a)

pddlstream.language.constants.Action

In [91]:
from pddlstream.language.conversion import value_from_evaluation

solution.certificate.all_facts

[('robotconf', Conf-Home),
 ('robotatconf', Conf-Home),
 ('canfreemove',),
 ('robottoolchangerempty',),
 ('element', 'b0'),
 ('atrack', 'b0'),
 ('iselement', 'b0'),
 ('grounded', 'b0'),
 ('element', 'b1'),
 ('atrack', 'b1'),
 ('iselement', 'b1'),
 ('grounded', 'b1'),
 ('element', 'b2'),
 ('atrack', 'b2'),
 ('iselement', 'b2'),
 ('element', 'b3'),
 ('atrack', 'b3'),
 ('iselement', 'b3'),
 ('element', 'b5'),
 ('atrack', 'b5'),
 ('iselement', 'b5'),
 ('grounded', 'b5'),
 ('element', 'b6'),
 ('atrack', 'b6'),
 ('iselement', 'b6'),
 ('element', 'b7'),
 ('atrack', 'b7'),
 ('iselement', 'b7'),
 ('grounded', 'b7'),
 ('element', 'b8'),
 ('atrack', 'b8'),
 ('iselement', 'b8'),
 ('element', 'b4'),
 ('atrack', 'b4'),
 ('iselement', 'b4'),
 ('joint', 'b0', 'b2'),
 ('joint', 'b2', 'b0'),
 ('notoolatjoint', 'b0', 'b2'),
 ('notoolatjoint', 'b2', 'b0'),
 ('joint', 'b0', 'b3'),
 ('joint', 'b3', 'b0'),
 ('notoolatjoint', 'b0', 'b3'),
 ('notoolatjoint', 'b3', 'b0'),
 ('joint', 'b0', 'b6'),
 ('joint', 'b6'

In [92]:
# action_instances = get_action_instances(task, plan)
is_valid_plan(task.init, solution.plan)

ValueError: Action(name='pick_gripper_from_rack', args=('g1', @conf158, @conf258, @traj60))

# Sequence

In [55]:
place_actions = [action for action in plan if action.name == 'place_element_on_structure']
element_sequence = [ac.args[0] for ac in actions]

In [13]:
process.assembly.sequence

['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']

In [14]:
element_sequence

['b0', 'b1', 'b2', 'b3', 'b5', 'b6', 'b7', 'b8', 'b4']

In [18]:
place_actions

[Action(name='place_element_on_structure', args=('b0', @conf133, @conf233, @traj35, 'g1')),
 Action(name='place_element_on_structure', args=('b1', @conf135, @conf235, @traj37, 'g1')),
 Action(name='place_element_on_structure', args=('b2', @conf137, @conf237, @traj39, 'g1')),
 Action(name='place_element_on_structure', args=('b3', @conf139, @conf239, @traj41, 'g1')),
 Action(name='place_element_on_structure', args=('b5', @conf141, @conf241, @traj43, 'g1')),
 Action(name='place_element_on_structure', args=('b6', @conf143, @conf243, @traj45, 'g1')),
 Action(name='place_element_on_structure', args=('b7', @conf145, @conf245, @traj47, 'g1')),
 Action(name='place_element_on_structure', args=('b8', @conf147, @conf247, @traj49, 'g1')),
 Action(name='place_element_on_structure', args=('b4', @conf149, @conf249, @traj51, 'g1'))]

# Visualize sequence in a pybullet window

In [27]:
from integral_timber_joints.planning.robot_setup import load_RFL_world
from integral_timber_joints.planning.run import set_initial_state

# * Connect to path planning backend and initialize robot parameters
# viewer or diagnosis or view_states or watch or step_sim,
client, robot, _ = load_RFL_world(viewer=1, verbose=False)
set_initial_state(client, robot, process, disable_env=False, reinit_tool=False)

In [37]:
for tool in process.clamps:
    print(process.clamp(tool.name))

CL3 (c1)
CL3 (c2)
CL3M (c3)
CL3M (c4)


In [41]:
for beam_id in process.assembly.sequence:
    print('='*5)
    print(beam_id)
    joint_ids = process.assembly.get_joint_ids_with_tools_for_beam(beam_id)
    for joint_id in joint_ids:
        print(process.assembly.get_joint_attribute(joint_id, 'tool_type'))

=====
b0
=====
b1
=====
b2
CL3M
CL3M
=====
b3
CL3M
CL3M
=====
b5
=====
b6
CL3
CL3
=====
b7
=====
b8
CL3
CL3
=====
b4
CL3
CL3


In [30]:
from integral_timber_joints.planning.pddlstream_definitions.compile_process_actions import actions_from_pddlstream_plan

actions_from_pddlstream_plan(process, plan, verbose=True)

Action(name='pick_gripper_from_rack', args=('g1', @conf126, @conf226, @traj27))
Action(name='pick_element_from_rack', args=('b0', @conf10, @conf20, @traj1, 'g1'))
Action(name='place_element_on_structure', args=('b0', @conf11, @conf21, @traj2, 'g1'))
Beam id: b0
|- Pick PG500 (g1) ('g1') from Storage
|- Operator load Beam ('b0') for pickup
|- Pick Beam ('b0') from PickupPoint using Gripper(g1)
|- Place Beam ('b0') to final location without clamps
Action(name='pick_element_from_rack', args=('b1', @conf12, @conf22, @traj3, 'g1'))
Action(name='place_element_on_structure', args=('b1', @conf13, @conf23, @traj4, 'g1'))
Beam id: b1
|- Operator load Beam ('b1') for pickup
|- Pick Beam ('b1') from PickupPoint using Gripper(g1)
|- Place Beam ('b1') to final location without clamps
Action(name='place_tool_at_rack', args=('g1', @conf127, @conf227, @traj28))
Action(name='pick_clamp_from_rack', args=('c2', @conf120, @conf220, @traj21))
Action(name='place_clamp_at_joint', args=('c2', 'b0', 'b2', @conf

Processing Seq 2 Action 0 Movement 0: Free Move to reach Pickup Approach Frame of Beam ('b2')
Processing Seq 2 Action 0 Movement 1: Gripper ('g1') Open Gripper before gripping Beam ('b2')
Processing Seq 2 Action 0 Movement 2: Linear Advance to Storage Frame of Beam ('b2')
Processing Seq 2 Action 0 Movement 3: Gripper ('g1') Close Gripper to grip Beam ('b2')
Processing Seq 2 Action 0 Movement 4: Linear Retract after picking up Beam ('b2')
Processing Seq 2 Action 0 Movement 0: Free Move to bring Beam ('b2') to approach clamps on structure.
Processing Seq 2 Action 0 Movement 1: Linear Advance to bring Beam ('b2') into clamp jaws
Processing Seq 2 Action 0 Movement 2: Clamps (['c2', 'c4']) close slightly to touch Beam ('b2')
Processing Seq 2 Action 0 Movement 3: Robot and Clamps (['c2', 'c4']) syncronously move to clamp Beam ('b2')
Processing Seq 2 Action 0 Movement 4: Open Gripper ('g1') and let go of Beam ('b2')
Processing Seq 2 Action 0 Movement 5: Linear retract after placing Beam ('b2'

Processing Seq 8 Action 0 Movement 1: Linear Advance to Storage Frame of PG500 (g1) ('g1'), to place tool in storage.
Processing Seq 8 Action 0 Movement 2: PG500 (g1) ('g1') Close Gripper to lock onto storage pad.
Processing Seq 8 Action 0 Movement 3: Toolchanger Unlock PG500 (g1) ('g1')
Processing Seq 8 Action 0 Movement 4: Linear Retract from storage after placing PG500 (g1) ('g1') in storage
Processing Seq 0 Action 0 Movement 0: Free Move to reach CL3 (c1) ('c1') to detach it from structure.
Processing Seq 0 Action 0 Movement 1: Linear Advance to mate toolchanger of CL3 (c1) ('c1') to detach it from structure.
Processing Seq 0 Action 0 Movement 2: Toolchanger Lock CL3 (c1) ('c1')
Processing Seq 0 Action 0 Movement 3: CL3 (c1) ('c1') Open Clamp Jaws to be released.
Processing Seq 0 Action 0 Movement 4: CL3 (c1) ('c1') Open Gripper to be released from structure.
Processing Seq 0 Action 0 Movement 5: Linear Retract 1 of 2 to storage after picking up CL3 (c1) ('c1') from structure.
Proc

1

In [32]:
client.disconnect()

In [31]:
from integral_timber_joints.planning.state import set_state
from integral_timber_joints.process import RoboticMovement, RobotClampAssemblyProcess

for m in process.movements:
    print(m.short_summary)
    start_state = process.get_movement_start_scene(m)
    end_state = process.get_movement_end_scene(m)  
    set_state(client, robot, process, end_state, initialize=False)
    pp.wait_if_gui('End state.')

RoboticFreeMovement(#A0_M0, Free Move reach Storage Approach Frame of PG500 (g1) ('g1'), to get tool., traj 0)
End state.
RoboticLinearMovement(#A0_M1, Linear Advance to Storage Frame of PG500 (g1) ('g1'), to get tool., traj 0)
End state.
RoboticDigitalOutput(#A0_M2, Toolchanger Lock PG500 (g1) ('g1'))
End state.
RoboticDigitalOutput(#A0_M3, PG500 (g1) ('g1') Open Gripper to release itself from storage pad.)
End state.
RoboticLinearMovement(#A0_M4, Linear Retract after getting PG500 (g1) ('g1') from storage., traj 0)
End state.
OperatorLoadBeamMovement(#A0_M0, Opeartor Load Beam (b0) to Pickup Location)
End state.
RoboticFreeMovement(#A0_M0, Free Move to reach Pickup Approach Frame of Beam ('b0'), traj 0)
End state.
RoboticDigitalOutput(#A0_M1, Gripper ('g1') Open Gripper before gripping Beam ('b0'))
End state.
RoboticLinearMovement(#A0_M2, Linear Advance to Storage Frame of Beam ('b0'), traj 0)
End state.
RoboticDigitalOutput(#A0_M3, Gripper ('g1') Close Gripper to grip Beam ('b0'))
E

RoboticDigitalOutput(#A0_M4, CL3M (c4) ('c4') Open Gripper to be released from structure.)
End state.
RoboticLinearMovement(#A0_M5, Linear Retract 1 of 2 to storage after picking up CL3M (c4) ('c4') from structure., traj 0)
End state.
RoboticLinearMovement(#A0_M6, Linear Retract 2 of 2 to storage after picking up CL3M (c4) ('c4') from structure., traj 0)
End state.
RoboticFreeMovement(#A0_M0, Free Move to bring CL3M (c4) ('c4') to structure., traj 0)
End state.
RoboticLinearMovement(#A0_M1, Linear Approach 1 of 2 to attach CL3M (c4) ('c4') to structure., traj 0)
End state.
RoboticLinearMovement(#A0_M2, Linear Approach 2 of 2 to attach CL3M (c4) ('c4') to structure., traj 0)
End state.
RoboticDigitalOutput(#A0_M3, CL3M (c4) ('c4') Close Gripper and attach to structure.)
End state.
RoboticDigitalOutput(#A0_M4, Toolchanger Unlock CL3M (c4) ('c4').)
End state.
RoboticLinearMovement(#A0_M5, Linear Retract after attaching CL3M (c4) ('c4') on structure, traj 0)
End state.
RoboticFreeMovement(

End state.
RoboticFreeMovement(#A0_M0, Free Move reach Storage Approach Frame of PG500 (g1) ('g1'), to get tool., traj 0)
End state.
RoboticLinearMovement(#A0_M1, Linear Advance to Storage Frame of PG500 (g1) ('g1'), to get tool., traj 0)
End state.
RoboticDigitalOutput(#A0_M2, Toolchanger Lock PG500 (g1) ('g1'))
End state.
RoboticDigitalOutput(#A0_M3, PG500 (g1) ('g1') Open Gripper to release itself from storage pad.)
End state.
RoboticLinearMovement(#A0_M4, Linear Retract after getting PG500 (g1) ('g1') from storage., traj 0)
End state.
OperatorLoadBeamMovement(#A0_M0, Opeartor Load Beam (b6) to Pickup Location)
End state.
RoboticFreeMovement(#A0_M0, Free Move to reach Pickup Approach Frame of Beam ('b6'), traj 0)
End state.
RoboticDigitalOutput(#A0_M1, Gripper ('g1') Open Gripper before gripping Beam ('b6'))
End state.
RoboticLinearMovement(#A0_M2, Linear Advance to Storage Frame of Beam ('b6'), traj 0)
End state.
RoboticDigitalOutput(#A0_M3, Gripper ('g1') Close Gripper to grip Bea

End state.
RoboticFreeMovement(#A0_M0, Free Move to reach CL3 (c1) ('c1') to detach it from structure., traj 0)
End state.
RoboticLinearMovement(#A0_M1, Linear Advance to mate toolchanger of CL3 (c1) ('c1') to detach it from structure., traj 0)
End state.
RoboticDigitalOutput(#A0_M2, Toolchanger Lock CL3 (c1) ('c1'))
End state.
ClampsJawMovement(#A0_M3, CL3 (c1) ('c1') Open Clamp Jaws to be released.)
End state.
RoboticDigitalOutput(#A0_M4, CL3 (c1) ('c1') Open Gripper to be released from structure.)
End state.
RoboticLinearMovement(#A0_M5, Linear Retract 1 of 2 to storage after picking up CL3 (c1) ('c1') from structure., traj 0)
End state.
RoboticLinearMovement(#A0_M6, Linear Retract 2 of 2 to storage after picking up CL3 (c1) ('c1') from structure., traj 0)
End state.
RoboticFreeMovement(#A0_M0, Free Move reach Storage Approach Frame of CL3 (c1) ('c1'), to place clamp in storage., traj 0)
End state.
RoboticLinearMovement(#A0_M1, Linear Approach 1 of 2 to place CL3 (c1) ('c1') in stor

End state.
ClampsJawMovement(#A0_M3, CL3 (c2) ('c2') Open Clamp Jaws to be released.)
End state.
RoboticDigitalOutput(#A0_M4, CL3 (c2) ('c2') Open Gripper to be released from structure.)
End state.
RoboticLinearMovement(#A0_M5, Linear Retract 1 of 2 to storage after picking up CL3 (c2) ('c2') from structure., traj 0)
End state.
RoboticLinearMovement(#A0_M6, Linear Retract 2 of 2 to storage after picking up CL3 (c2) ('c2') from structure., traj 0)
End state.
RoboticFreeMovement(#A0_M0, Free Move reach Storage Approach Frame of CL3 (c2) ('c2'), to place clamp in storage., traj 0)
End state.
RoboticLinearMovement(#A0_M1, Linear Approach 1 of 2 to place CL3 (c2) ('c2') in storage., traj 0)
End state.
RoboticLinearMovement(#A0_M2, Linear Approach 2 of 2 to place CL3 (c2) ('c2') in storage., traj 0)
End state.
RoboticDigitalOutput(#A0_M3, Close Gripper to lock CL3 (c2) ('c2') onto storage pad.)
End state.
RoboticDigitalOutput(#A0_M4, Toolchanger Unlock CL3 (c2) ('c2'))
End state.
RoboticLine

# Misc

In [25]:
# from pddlstream.algorithms.algorithm import parse_problem
# from pddlstream.algorithms.constraints import PlanConstraints
# from pddlstream.algorithms.downward import get_problem, task_from_domain_problem
# from pddlstream.algorithms.meta import examine_instantiated

# pddlstream_problem = get_pddlstream_problem(process, use_partial_order=True, debug=False,
#                                             reset_to_home=False)

# print(pddlstream_problem.goal)

# results, instantiated = examine_instantiated(pddlstream_problem, unit_costs=1, verbose=0, debug=1)